In [8]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=6

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

#function to get a random quadratic polynomial of variables x(T)
function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],0:2))
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# ball constraints on subsets of variables
u=3# clique size
p=floor(Int64,n/u) #number of cliques

# indices of variables on each clique
I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u 
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
vecf=[generate_random_poly(I[j]) for j in 1:p] #vector of separable polynomials on each clique
f=sum(vecf)

# ball constraints on each clique
g=[1.0-sum(x[I[j]].^2) for j in 1:p]
J=[j:j for j in 1:p] # assign inequality constraints


m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")


l=0#ceil(Int64, n/7)# number of the equality constraints

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]# assign equality constraints
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{true,Float64}}(undef,l)


# get a random point satisfies the inequality constraints
randx=2*rand(Float64,n).-1

for j in 1:p
    randx[I[j]]=randx[I[j]]./norm(randx[I[j]])
    randx[I[j]]=rand(Float64,1)[1]*randx[I[j]]
end

for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make the random point satisfy the equality constraint h[i](randx) = 0
    end
end

l=length(h)
println("Number of equality constraints: l=",l)


***Problem setting***
Number of variable: n=6
Number of inequality constraints: m=2
Number of equality constraints: l=0


In [9]:
using ctpPOP

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=ctpPOP.get_info(x,f,g,h,sparse=true);

In [10]:
k=2

println("Relaxed order: k=",k)

Relaxed order: k=2


In [11]:
using ctpPOP

opt_val1=ctpPOP.POP_CS_CGAL(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
    maxit=Int64(1e10),tol=1e-2,use_eqcons_to_get_constant_trace=false,check_tol_each_iter=true)

------------------------------------------------------
The clique sizes of varibles:
[4, 3]
[1, 1]
------------------------------------------------------
  Number of cliques: p=2
  Largest clique size: u=4
  Largest constant trace: a=3.0
  Number of blocks: omega=4
  Number of equality consraints: zeta=101
  Size of the largest block: s^max=15
Modeling time:
  0.010849 seconds (17.95 k allocations: 1.562 MiB)
iter=1   val=-3.5841484348982773   gap=0.06801841349034697   feas=3.6759107994647753
iter=2   val=-1.1344630298955933   gap=0.696194168326104   feas=0.9018136575147957
iter=4   val=-1.4364846496163075   gap=0.2620683937125096   feas=0.7532263142370321
iter=8   val=-1.5549121880417505   gap=0.10379167018572825   feas=0.1667305967659999
iter=16   val=-1.70979570300986   gap=0.026675277520596497   feas=0.04003599616829153
iter=32   val=-1.7554389917169235   gap=0.0066882534423136734   feas=0.010153204776730073
iter=33   val=-1.7563416145544202   gap=0.006307692476822366   feas=0.0095

-1.7563416145544202

In [12]:
using ctpPOP

opt_val1=ctpPOP.POP_CS_LMBM(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh,k,
    tol=1e-2,use_eqcons_to_get_constant_trace=false)

------------------------------------------------------
The clique sizes of varibles:
[4, 3]
[1, 1]
------------------------------------------------------
  Number of cliques: p=2
  Largest clique size: u=4
  Largest constant trace: a=3.0
  Number of blocks: omega=4
  Number of equality consraints: zeta=101
  Size of the largest block: s^max=15
Modeling time:
  0.010509 seconds (17.95 k allocations: 1.562 MiB)
**LMBM solver:
---------------
| Parameters: |
---------------
n:       101
maxtime: 300000.000000
na:      2
mcu:     5
mc:      7
rpar: 
ipar: 
 Entry to LMBM:
 NIT=    1  NFE=    1  F= 0.51115264D+00  WK= 0.4125D-01  QK= 0.2063D-01
 NIT=    2  NFE=    2  F= 0.49610278D+00  WK= 0.4125D-01  QK= 0.2063D-01
 NIT=    3  NFE=    3  F= 0.48892040D+00  WK= 0.4125D-01  QK= 0.2063D-01
 NIT=    4  NFE=    6  F= 0.48859910D+00  WK= 0.4125D-01  QK= 0.2063D-01
 NIT=    5  NFE=   46  F= 0.48859910D+00  WK= 0.4125D-01  QK= 0.2063D-01
 Exit from LMBM:
 Abnormal exit: The value of the function d

-3.4787208568690855

In [15]:
using TSSOS


@time opt,sol,data=cs_tssos_first(Vector{SparseMatrixCSC{UInt8,UInt32}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,CS="MD",TS=false);

------------------------------------------------------
The clique sizes of varibles:
[4, 3]
[1, 1]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 100             
  Cones                  : 0               
  Scalar variables       : 1               
  Matrix variables       : 4               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 

In [14]:
#using ctpPOP

#optval=ctpPOP.POP_NLP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)